In [1]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G = Generator(16, 256, 512, 16).eval().to(device)

g_checkpoint = None
if torch.cuda.is_available():
    g_checkpoint = torch.load('autovc.ckpt')
else :
    g_checkpoint = torch.load('autovc.ckpt', map_location='cpu')
G.load_state_dict(g_checkpoint)

metadata = pickle.load(open('metadata.pkl', "rb"))

spect_vc = []

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        print(uttr_org.shape, emb_org.shape)
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)          

torch.Size([1, 544, 80]) torch.Size([1, 256])
torch.Size([1, 544, 80]) torch.Size([1, 256])


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/suzukidaishi/.pyenv/versions/3.8.6/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-0eff20e03ae1>", line 41, in <module>
    _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
  File "/Users/suzukidaishi/.pyenv/versions/3.8.6/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/Users/suzukidaishi/Desktop/autovc/model_vc.py", line 194, in forward
    mel_outputs = self.decoder(encoder_outputs)
  File "/Users/suzukidaishi/.pyenv/versions/3.8.6/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/Users/suzukidaishi/Desktop/autovc/model_vc.py", line 118, in forward
    outputs, _ = self.lstm2(x)
  File "/Users/suzukidaishi/.pyenv/versions/3.8.6/lib/python3.8/site-pa

TypeError: object of type 'NoneType' has no len()